In [2]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import math

In [6]:
import sklearn
from scipy import linalg
import sklearn.discriminant_analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KernelDensity as parzen

In [7]:
def read_data():
    f=open("iris(150).csv", 'r')
    lines=[line.strip() for line in f.readlines()]
    f.close()
    
    lines=[line.split(",") for line in lines if line]
    #print('cool guy', lines)
    lines = lines[1:]
    #print(lines)
    #print([line[:6] for line in lines if line])
    
        
    data=np.array([line[1:5] for line in lines if line], dtype=np.float)
    
    class1=np.array([line[1:5] for line in lines if line[-1]=="setosa"], dtype=np.float)
    
    class2=np.array([line[1:5] for line in lines if line[-1]=="virginica"], dtype=np.float)
    
    class3=np.array([line[1:5] for line in lines if line[-1]=="versicolor"], dtype=np.float)
    
    #list of class labels
    labels=[]
    for line in lines:
        #print(line)
        strt=line.pop()
        #print(strt)
        labels.append(strt)
    
    #print(labels)
    #create array of labels
    labels=[line.split(",") for line in labels if line]
    t=np.zeros(shape=(150, 3))
    t_n = np.zeros(shape=(150,1))
    #create target vector encoded according to 1-of-K scheme
    for i in range(len(data)):
        if labels[i]==["setosa"]: 
            t[i][0]=1
            t_n[i] = 0
        elif labels[i]==["versicolor"]: 
            t[i][1]=1
            t_n[i] = 1
        elif labels[i]==["virginica"]: 
            t[i][2]=1
            t_n[i] = 2
    
    return class1, class2, class3, data, t, t_n


In [8]:
def gaussian(x, mean, cov):
    xm=np.reshape((x-mean), (-1, 1))
    px=1/(math.pow(2.0*math.pi, 2))*1/math.sqrt(np.linalg.det(cov))*math.exp(-(np.dot(np.dot(xm.T, np.linalg.inv(cov)), xm))/2)
    return px

In [14]:
def main():
    class1, class2, class3, data, t, t_n=read_data()

    count=np.zeros(shape=(150,1))
    t_assigned=np.zeros(shape=(150, 3))
    cov=np.zeros(shape=(3, 4, 4))
    mean=np.zeros(shape=(3, 4))

    #compute means for each class
    mean1=class1.mean(axis=0)
    mean2=class2.mean(axis=0)
    mean3=class3.mean(axis=0)
    #compute covariance matrices, such that the columns are variables and rows are observations of variables
    cov1=np.cov(class1, rowvar=0)
    cov2=np.cov(class2, rowvar=0)
    cov3=np.cov(class3, rowvar=0)
    
    #compute gaussian likelihood functions p(x|Ck) for each class
    for i in range(135, 150):
        px1=(1/3.0)*gaussian(data[i], mean1, cov1)
        px2=(1/3.0)*gaussian(data[i], mean2, cov2)
        px3=(1/3.0)*gaussian(data[i], mean3, cov3)
        m=np.max([px1, px2, px3])
    #compute posterior probability p(Ck|x) assuming that p(x|Ck) is gaussian and the entire expression is wrapped by sigmoid function 
        pc1=((math.exp(px1)*math.exp(-m))*math.exp(m))/((math.exp(px2)*math.exp(-m)+math.exp(px3)*math.exp(-m))*math.exp(m))
        pc2=((math.exp(px2)*math.exp(-m))*math.exp(m))/((math.exp(px1)*math.exp(-m)+math.exp(px3)*math.exp(-m))*math.exp(m))
        pc3=((math.exp(px3)*math.exp(-m))*math.exp(m))/((math.exp(px1)*math.exp(-m)+math.exp(px2)*math.exp(-m))*math.exp(m))
        #assign p(Ck|x)=1 if p(Ck|x)>>p(Cj|x) for all j!=k
        if pc1>pc2 and pc1>pc3: t_assigned[i][0]=1
        elif pc3>pc1 and pc3>pc2: t_assigned[i][1]=1
        elif pc2>pc1 and pc2>pc3: t_assigned[i][2]=1
    #count the number of misclassifications
        for j in range(3):
            if t[i][j]-t_assigned[i][j]!=0: count[i]=1

    cov=[cov1, cov2, cov3]
    mean=[mean1, mean2, mean3]

    t1=np.zeros(shape=(len(class1), 1))
    t2=np.zeros(shape=(len(class2), 1))
    t3=np.zeros(shape=(len(class3), 1))
    for i in range(len(data)):
        for j in range(len(class1)):
            if t_assigned[i][0]==1: t1[j]=1
            elif t_assigned[i][1]==1: t2[j]=2
            elif t_assigned[i][2]==1: t3[j]=3


    #print ("number of misclassifications", sum(count) )#, "assigned labels to data points", t_assigned, "target data", t)
    
    """
    #print(t)
    qda = QDA(store_covariances=True)
    #qda = LDA(store_covariances=True)
    start = 0
    end = 15
    y_pred = qda.fit(data, t_n).predict(data[start:end])
    cnt = 0
    for i in range(15):
        if y_pred[i] != t_n[start+i][0]:
            cnt+=1
    #print(cnt)
    
    #print(t)
    qda = QDA(store_covariances=True)
    #qda = LDA(store_covariances=True)
    start = 0
    end = 15
    y_pred = qda.fit(data, t_n).predict(data[start:end])
    cnt = 0
    for i in range(15):
        if y_pred[i] != t_n[start+i][0]:
            cnt+=1
    #print(cnt)
    """
    kde = parzen(bandwidth = 0.01, kernel='gaussian')
    start = 0
    end = 15
    y_pred = kde.fit(data, t_n).score_samples(data[start:end]) #.predict(data[start:end])
    cnt = 0
    
    print(y_pred)
    """for i in range(15):
        if y_pred[i] != t_n[start+i][0]:
            cnt+=1
    print(cnt)
"""

main()

[ 9.73429132  9.73429132  9.73429132  9.73429132  9.73429132  9.73429132
  9.73429132  9.73429132  9.73429132  9.73429132  9.73429132  9.73429132
  9.73429132  9.73429132  9.73429132]
